# National Water Model Benchmark Usage
**NOTE**: 
_This notebook adapted from originals by Timothy Hodson and Rich Signell. See that upstream work at:_
* https://github.com/thodson-usgs/dscore
* https://github.com/USGS-python/hytest-evaluation-workflows/

This notebook will demonstrate how to call the specific functions defined in the [NWM Standard Suite v1](./NWM_StandardSuite_v1.ipynb)
notebook, using a small demonstration dataset.

In [1]:
import pandas as pd
import numpy as np

## Sample Data

In [2]:
ds = pd.read_csv(r"./NWM_Benchmark_SampleData.csv", index_col='date').dropna()
print(len(ds.index), " Records")
ds.head()

12145  Records


,site_no,obs,nwm,nhm
date,,,,
1983-10-01,1104200,1.121347,6.175417,1.469472
1983-10-02,1104200,1.214793,6.250417,1.848861
1983-10-03,1104200,0.872159,6.215833,2.169456
1983-10-04,1104200,0.419089,6.105000,2.200083
1983-10-05,1104200,0.849505,5.952500,1.931588


## Import Benchmark Functions
The benchmark functions are defined in an [adjacent notebook](./NWM_StandardSuite_v1.ipynb).  They are imported here by 
running that notebook from within the following cell:

In [4]:
%run ./NWM_StandardSuite_v1.ipynb
# This brings functions defined in external notebook into this notebook's namespace.

The functions are now available here, to run against our sample data:

In [5]:
# Mean Square Error
MSE(obs=ds['obs'], sim=ds['nwm'])

55.73589185136414

In [6]:
# Kling-Gupta efficiency
KGE(obs=ds['obs'], sim=ds['nwm'])

0.3286282290159983

## Create Composite Benchmark
It is useful to combine several of these metrics into a single benchmark routine, which returns a 
pandas [Series](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.html) of 
the assembled metrics.

In [7]:
def compute_benchmark(ds):
    obs = ds['obs']
    sim = ds['nwm']
    return pd.Series(
        data={
            'NSE': NSE(obs, sim),
            'KGE': KGE(obs, sim),
            'logNSE': logNSE(obs, sim),
            'pbias': pbias(obs, sim),
            'rSD': rSD(obs, sim),
            'pearson': pearson_r(obs, sim),
            'spearman': spearman_r(obs, sim), 
            'pBiasFMS': pBiasFMS(obs, sim),
            'pBiasFLV': pBiasFLV(obs, sim),
            'pBiasFHV': pBiasFHV(obs, sim)
        },
        name=ds['site_no'][0], # special case -- 'site_no' column
        dtype='float64'
    )

In [8]:
compute_benchmark(ds)

NSE          0.002319
KGE          0.328628
logNSE       0.184820
pbias       52.561009
rSD          1.375604
pearson      0.817255
spearman     0.804573
pBiasFMS   -34.136547
pBiasFLV    81.492827
pBiasFHV    32.058168
Name: 1104200, dtype: float64